In [1]:
from pyntcloud import PyntCloud
from pyntcloud.io import bin as io_bin
import numpy as np
import pandas as pd
import os

from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text, Output, VBox
from traitlets import link, dlink

In [2]:
VELODYNE_HEIGHT = 1.73

def extractBB(label,gt=True):
    label_split = label.split(" ")
    if gt:
        return label_split[0],float(label_split[-7]),float(label_split[-6]),float(label_split[-5]),float(label_split[-4]),float(label_split[-3]),float(label_split[-2]),float(label_split[-1])
    else:
        return label_split[0],float(label_split[-8]),float(label_split[-7]),float(label_split[-6]),float(label_split[-5]),float(label_split[-4]),float(label_split[-3]),float(label_split[-2])
    
def getCorners(height,width,length,x,y,z,θ,rotation=True):
    
    corners = np.array([[-length / 2, -length / 2, length / 2, length / 2, -length / 2, -length / 2, length / 2, length / 2],
                        [width / 2, -width / 2, -width / 2, width / 2, width / 2, -width / 2, -width / 2, width / 2],
                        [0, 0, 0, 0, height, height, height, height]])
    
    rotMat = np.array([[np.cos(θ) , -np.sin(θ) , 0],
                       [np.sin(θ) ,  np.cos(θ) , 0],
                       [    0     ,     0      , 1]])
    if rotation:
        cornersPos = (np.dot(rotMat,corners)+np.tile([x,y,z],(8,1)).T).transpose()
        corner1,corner2,corner3,corner4,corner5,corner6,corner7,corner8 = cornersPos[0],cornersPos[1],cornersPos[2],cornersPos[3],cornersPos[4],cornersPos[5],cornersPos[6],cornersPos[7]
    else:
        cornersPos = (corners + np.tile([x,y,z],(8,1)).T).transpose()
        corner1,corner2,corner3,corner4,corner5,corner6,corner7,corner8 = cornersPos[0],cornersPos[1],cornersPos[2],cornersPos[3],cornersPos[4],cornersPos[5],cornersPos[6],cornersPos[7]
    
    return list(corner1),list(corner2),list(corner3),list(corner4),list(corner5),list(corner6),list(corner7),list(corner8)

def createBBox(bounding_box,C1,C2,C3,C4,C5,C6,C7,C8,color="yellow"):
    bounding_box.append(
        {
            "color":color,
            "vertices":[C1,C2,C3,C4,C1]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C1,C4,C8,C5,C1]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C1,C2,C6,C5,C1]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C2,C6,C7,C3,C2]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C3,C7,C8,C4,C3]
        })
    return bounding_box

In [3]:
KITTI_PATH = path+ "/data/KITTI/object/training/"
PointRCNN_PATH2 = path+ "/PointRCNN/pretrained_model.pth"
PointRCNN_PATH = path+ "PointRCNN/resultdata/data/"

PC_ID = '%06d' % 2300 # [0 : 7480] 

points = io_bin.read_bin(KITTI_PATH+"velodyne/"+PC_ID+".bin",shape=(-1,4))['points']
points['z'] += VELODYNE_HEIGHT

assert os.path.exists(KITTI_PATH+"label_2/"+PC_ID+".txt"), "File "+KITTI_PATH+"label_2/"+PC_ID+".txt doesn't exist !"
file_label_gt = open(KITTI_PATH+"label_2/"+PC_ID+".txt","r")
labels_gt = file_label_gt.readlines()
file_label_gt.close()

assert os.path.exists(PointRCNN_PATH+PC_ID+".txt"), "File "+PointRCNN_PATH+PC_ID+".txt doesn't exist !"
file_label_pred = open(PointRCNN_PATH+PC_ID+".txt","r")
labels_pred = file_label_pred.readlines()
file_label_pred.close()

labels_clean_gt = []
for i,label in enumerate(labels_gt):
    labels_gt[i]=label[:-2]
    if labels_gt[i].split(" ")[0] in ["Car", "Van", "Truck"]:
        labels_clean_gt.append(extractBB(labels_gt[i],gt=True))

labels_clean_pred = []
for i,label in enumerate(labels_pred):
    labels_pred[i]=label[:-2]
    if labels_pred[i].split(" ")[0] == "Car":
        labels_clean_pred.append(extractBB(labels_pred[i],gt=False))

bounding_box = []
colors = {"GT":'blue',"Pred":"yellow"}

boxes_gt = list()
for label in labels_clean_gt: 
    object_type,height,width,length,x_tmp,y_tmp,z_tmp,θ = label
    x,y,z,θ = z_tmp,-x_tmp,y_tmp-VELODYNE_HEIGHT,np.pi/2-θ
    C1,C2,C3,C4,C5,C6,C7,C8 = getCorners(height,width,length,x,y,z,θ,rotation=True)
    bounding_box = createBBox(bounding_box,C1,C2,C3,C4,C5,C6,C7,C8,colors["GT"])
    boxes_gt.append(list((getCorners(height,width,length,x,y,z,θ,rotation=True))))
    
boxes_pred = list()
for label in labels_clean_pred: 
    object_type,height,width,length,x_tmp,y_tmp,z_tmp,θ = label
    x,y,z,θ = z_tmp,-x_tmp,y_tmp-VELODYNE_HEIGHT,np.pi/2-θ
    C1,C2,C3,C4,C5,C6,C7,C8 = getCorners(height,width,length,x,y,z,θ,rotation=True)
    bounding_box = createBBox(bounding_box,C1,C2,C3,C4,C5,C6,C7,C8,colors["Pred"])
    boxes_pred.append(list((getCorners(height,width,length,x,y,z,θ,rotation=True))))

    
cloud = PyntCloud(points)
cloud.plot(initial_point_size=0.02, polylines=bounding_box)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(0.007973304018378258, 36.71273982524872, 7.7…

In [6]:
file_label_gt = open(KITTI_PATH+"label_2/"+PC_ID+".txt","r")
labels_gt = file_label_gt.readlines()
labels_gt

['Car 0.98 0 2.29 0.00 228.49 332.57 374.00 1.42 1.57 4.10 -2.83 1.80 3.04 1.59\n',
 'Car 0.00 0 1.66 499.50 186.11 555.36 231.99 1.38 1.53 3.58 -2.73 1.84 24.16 1.55\n',
 'Car 0.00 0 -1.67 658.75 182.85 709.34 222.44 1.49 1.65 4.06 2.95 1.92 29.78 -1.57\n',
 'Car 0.00 1 -1.52 550.60 183.78 580.17 210.69 1.48 1.54 4.05 -2.63 2.15 42.73 -1.58\n',
 'DontCare -1 -1 -10 585.25 177.30 647.64 197.65 -1 -1 -1 -1000 -1000 -1000 -10\n']

In [7]:
labels_clean_gt

[('Car', 1.42, 1.57, 4.1, -2.83, 1.8, 3.04, 1.5),
 ('Car', 1.38, 1.53, 3.58, -2.73, 1.84, 24.16, 1.5),
 ('Car', 1.49, 1.65, 4.06, 2.95, 1.92, 29.78, -1.5),
 ('Car', 1.48, 1.54, 4.05, -2.63, 2.15, 42.73, -1.5)]

In [7]:
def draw_simple(pc):
    fig = mlab.figure(figure=None, bgcolor=(0,0,0), fgcolor=None, engine=None, size=(1600, 1000))
    mlab.points3d(pc[:,0], pc[:,1], pc[:,2], mode='point', colormap = 'gnuplot', scale_factor=1, figure=fig)
    return fig

def draw_gt_boxes3d(gt_boxes3d, fig, thres = 0, color=(1,1,1), line_width=1, draw_text=True, text_scale=(1,1,1), color_list=None):

    num = len(gt_boxes3d)

    for n in range(num):
        b = gt_boxes3d[n]
        if color_list is not None:
            color = color_list[n] 
        if n >= thres:
            color = (0,1,0)
        else:
            color = (1,0,0)
        if draw_text: mlab.text3d(b[4,0], b[4,1], b[4,2], '%d'%n, scale=text_scale, color=color, figure=fig)
        for k in range(0,4):
            #http://docs.enthought.com/mayavi/mayavi/auto/mlab_helper_functions.html
            i,j=k,(k+1)%4
            mlab.plot3d([b[i,0], b[j,0]], [b[i,1], b[j,
                                                     1]], [b[i,2], b[j,2]], color=color, tube_radius=None, line_width=line_width, figure=fig)

            i,j=k+4,(k+1)%4 + 4
            mlab.plot3d([b[i,0], b[j,0]], [b[i,1], b[j,1]], [b[i,2], b[j,2]], color=color, tube_radius=None, line_width=line_width, figure=fig)

            i,j=k,k+4
            mlab.plot3d([b[i,0], b[j,0]], [b[i,1], b[j,1]], [b[i,2], b[j,2]], color=color, tube_radius=None, line_width=line_width, figure=fig)
    mlab.show()
    return fig
    #mlab.view(azimuth=180, elevation=70, focalpoint=[ 12.0909996 , -1.04700089, -2.03249991], distance=62.0, figure=fig)
    


In [13]:
all_box = list()
#for each in range(len(bounding_box)):
all_box.append(boxes_gt)
all_box.append(boxes_pred)

In [15]:
from mayavi import mlab
import numpy as np

c = np.random.rand(200, 3)
r = np.random.rand(200) / 100.

mlab.points3d(c[:1, 0], c[:1, 1], c[:1, 2], r[:1], color=(0.23, 0.4, 0.5))

#mlab.points3d(c[50:, 0], c[50:, 1], c[50:, 2], r[50:], color=(0.13, 0.35, 0.1235))
#np.repeat((0.13, 0.35, 0.1235), 2, axis=0)
#draw_lidar_coloured(c, (0.13, 0.35, 0.1235),1)
#draw_lidar_coloured(0, (0.13, 0.35, 0.1235),0)

mlab.show()

In [14]:
from mayavi import mlab
#mlab.clf
fig = draw_simple(points.to_numpy().astype(float))
#fig = draw_lidar_colored(points.to_numpy().astype(float))
thres = len(boxes_gt)
all_boxes = np.concatenate(all_box)
fig = draw_gt_boxes3d(all_boxes, fig,thres = thres,color=(0,1,0))
#draw_gt_boxes3d(np.asarray(boxes_pred), fig,color=(1,0,0))

In [ ]:
from mayavi import mlab
mlab.close()